In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
tb_data = './data/'

##### M1 = 9447 ~ 9477 / M2 = 9477 ~ 9507 / M3 = 9507 ~ 9537 / M4 = 9537 ~ 9567

### **1. All : 處理基本屬性資料**

In [18]:
CIF = os.path.join(tb_data, 'TBN_CIF.csv')
cif_use = pd.read_csv(CIF)
cif_use.drop(['CUST_START_DT'], axis = 1, inplace = True)
cif_use['GENDER_CODE'] = cif_use['GENDER_CODE'].map({'F': 1, 'M': 0})

In [19]:
cif_use.reset_index(drop=True, inplace=True)
print(cif_use.info())
print('---------')
print(cif_use.isnull().any())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187679 entries, 0 to 187678
Data columns (total 7 columns):
CUST_NO              187679 non-null object
AGE                  187679 non-null int64
CHILDREN_CNT         187247 non-null float64
EDU_CODE             158691 non-null float64
GENDER_CODE          185685 non-null float64
INCOME_RANGE_CODE    187297 non-null float64
WORK_MTHS            151489 non-null float64
dtypes: float64(5), int64(1), object(1)
memory usage: 10.0+ MB
None
---------
CUST_NO              False
AGE                  False
CHILDREN_CNT          True
EDU_CODE              True
GENDER_CODE           True
INCOME_RANGE_CODE     True
WORK_MTHS             True
dtype: bool


In [16]:
cif_use['GENDER_CODE'] = cif_use['GENDER_CODE'].map({'F': 1, 'M': 0})
m_edu = round(cif_use.loc[:,'EDU_CODE'].mean())
m_income = round(cif_use.loc[:, 'INCOME_RANGE_CODE'].mean())
m_work = round(cif_use.loc[:, 'WORK_MTHS'].mean())
name_ = cif_use.columns.tolist()[2:7]
mean_ = [0, m_edu, 2, m_income, m_work] 
for i,j in zip(name_, mean_):
    cif_use.loc[:,i] = cif_use.loc[:, i].fillna(j)

In [17]:
cif_use.head()

,CUST_NO,AGE,CHILDREN_CNT,EDU_CODE,GENDER_CODE,INCOME_RANGE_CODE,WORK_MTHS
0,UFUIMEGMK2KTIWI4,1,0.0,4.0,2.0,1.0,1.0
1,3HJN2X_LGBHBKILI,3,0.0,4.0,2.0,1.0,1.0
2,MNEZK2O3OE-JHYBW,4,0.0,3.0,2.0,2.0,2.0
3,W7ADZSWYXEMHB7JQ,4,0.0,5.0,2.0,2.0,1.0
4,ODB6BA6ORHL2UYZE,4,0.0,3.0,2.0,2.0,1.0


In [19]:
cif_use.shape

(187679, 7)

In [8]:
cif_use.to_csv('./data/cif_use.csv', index = False)

### **2.1 Train_use :**
#### **篩出Train所需之各個商品30天內有無購買(30~60天, m3)**

In [10]:
## 外匯
FX = os.path.join(tb_data, 'TBN_FX_TXN.csv')
tb_fx = pd.read_csv(FX)
tb_fx.rename(columns={'TXN_DT':'FX_DT'}, inplace = True)
tb_fx['FX_30'] = [1 if 9507 < i <= 9537 else 0 for i in tb_fx['FX_DT']]
fx_m3 = tb_fx.loc[tb_fx['FX_30'] == 1,['CUST_NO', 'FX_30']].drop_duplicates(subset = 'CUST_NO')
fx_m3.reset_index(drop=True, inplace=True)
fx_m3.to_csv('./data/fx_m3.csv', index = False)

In [11]:
## 信用卡
tb_cc = pd.read_csv('./data/TBN_CC_APPLY.csv')
tb_cc.rename(columns={'TXN_DT':'CC_DT'}, inplace = True)
tb_cc['CC_30'] = [1 if 9507 < i <= 9537 else 0 for i in tb_cc['CC_DT']]
cc_m3 = tb_cc.loc[tb_cc['CC_30'] == 1,['CUST_NO', 'CC_30']].drop_duplicates(subset = 'CUST_NO')
cc_m3.reset_index(drop = True, inplace = True)
cc_m3.to_csv('./data/cc_m3.csv', index = False)

In [12]:
## 信貸
tb_ln = pd.read_csv('./data/TBN_LN_APPLY.csv')
tb_ln.rename(columns={'TXN_DT':'LN_DT'}, inplace = True)
tb_ln['LN_30'] = [1 if 9507 < i <= 9537 else 0 for i in tb_ln['LN_DT']]
ln_m3 = tb_ln.loc[tb_ln['LN_30'] == 1,['CUST_NO', 'LN_30']].drop_duplicates(subset = 'CUST_NO')
ln_m3.reset_index(drop = True, inplace = True)
ln_m3.to_csv('./data/ln_m3.csv', index = False)

In [13]:
## 信託
tb_wm = pd.read_csv('./data/TBN_WM_TXN.csv')
tb_wm.rename(columns={'TXN_DT':'WM_DT'}, inplace = True)
tb_wm['WM_30'] = [1 if 9507 < i <= 9537 else 0 for i in tb_wm['WM_DT']]
wm_m3 = tb_wm.loc[tb_wm['WM_30'] == 1,['CUST_NO', 'WM_30']].drop_duplicates(subset = 'CUST_NO')
wm_m3.reset_index(drop = True, inplace = True)
wm_m3.to_csv('./data/wm_m3.csv', index = False)

### **2.2 Y_Zero_use :** 
#### **篩出Predict所需之各個商品30天內有無購買(0~30天, m4)**

In [15]:
## 外匯
FX = os.path.join(tb_data, 'TBN_FX_TXN.csv')
tb_fx = pd.read_csv(FX)
tb_fx.rename(columns={'TXN_DT':'FX_DT'}, inplace = True)
tb_fx['FX_30'] = [1 if 9537 < i <= 9567 else 0 for i in tb_fx['FX_DT']]
fx_m4 = tb_fx.loc[tb_fx['FX_30'] == 1,['CUST_NO', 'FX_30']].drop_duplicates(subset = 'CUST_NO')
fx_m4.reset_index(drop=True, inplace=True)
fx_m4.to_csv('./data/fx_m4.csv', index = False)

In [16]:
## 信用卡
tb_cc = pd.read_csv('./data/TBN_CC_APPLY.csv')
tb_cc.rename(columns={'TXN_DT':'CC_DT'}, inplace = True)
tb_cc['CC_30'] = [1 if 9537 < i <= 9567 else 0 for i in tb_cc['CC_DT']]
cc_m4 = tb_cc.loc[tb_cc['CC_30'] == 1,['CUST_NO', 'CC_30']].drop_duplicates(subset = 'CUST_NO')
cc_m4.reset_index(drop = True, inplace = True)
cc_m4.to_csv('./data/cc_m4.csv', index = False)

In [17]:
## 信貸
tb_ln = pd.read_csv('./data/TBN_LN_APPLY.csv')
tb_ln.rename(columns={'TXN_DT':'LN_DT'}, inplace = True)
tb_ln['LN_30'] = [1 if 9537 < i <= 9567 else 0 for i in tb_ln['LN_DT']]
ln_m4 = tb_ln.loc[tb_ln['LN_30'] == 1,['CUST_NO', 'LN_30']].drop_duplicates(subset = 'CUST_NO')
ln_m4.reset_index(drop = True, inplace = True)
ln_m4.to_csv('./data/ln_m4.csv', index = False)

In [18]:
## 信託
tb_wm = pd.read_csv('./data/TBN_WM_TXN.csv')
tb_wm.rename(columns={'TXN_DT':'WM_DT'}, inplace = True)
tb_wm['WM_30'] = [1 if 9537 < i <= 9567 else 0 for i in tb_wm['WM_DT']]
wm_m4 = tb_wm.loc[tb_wm['WM_30'] == 1,['CUST_NO', 'WM_30']].drop_duplicates(subset = 'CUST_NO')
wm_m4.reset_index(drop = True, inplace = True)
wm_m4.to_csv('./data/wm_m4.csv', index = False)

### **3.1 Train_use: 處理網址資料**
##### **篩出m3網址瀏覽之資料>10000**

In [21]:
tb_data = './data/'
cust = os.path.join(tb_data, 'TBN_CUST_BEHAVIOR.csv')
tb_web = pd.read_csv(cust)
tb_web['PAGE_3'] = tb_web['PAGE'].str.split('/').str.get(3)
tb_web['PAGE_4'] = tb_web['PAGE'].str.split('/').str.get(4)
tb_web['PAGE_M'] = tb_web['PAGE_3'] +'/' + tb_web['PAGE_4']

In [22]:
#篩出30~60天(m3)的瀏覽資料,擷取網址
web_m3 = tb_web[np.logical_and(tb_web['VISITDATE'] <= 9537, tb_web['VISITDATE'] > 9507)]
web_m3.reset_index(drop = True, inplace = True)

##### **篩m3網址瀏覽次數>10000**

In [23]:
#做網址dummy，合併cust_no,計算客戶單一網址瀏覽次數
web_dummy_m3 = pd.get_dummies(web_m3['PAGE_M'])
#sum the columns
row_m3 = {col: web_dummy_m3[col].sum() for col in web_dummy_m3}
# Turn the sums into a DataFrame with one row with an index of 'Total':
sum_m3 = pd.DataFrame(row_m3, index=['Total'])
# Now append the row:
web_dummy_m3 = web_dummy_m3.append(sum_m3)
# select the page columns which browse more than 10000
web_m3_select = web_dummy_m3.loc[:,web_dummy_m3.loc['Total'] >= 10000]
#rename page
page_m3 = web_m3_select.columns.tolist()
m3_list = []
for i in range(len(web_m3_select.columns)):
    m3_list.append('web_' + str(i+1))
web_m3_select.columns = m3_list

In [24]:
page_pare = pd.DataFrame({'Page':page_m3, 'No.':m3_list})
page_pare.head(7)

,Page,No.
0,edrn/deoxt,web_1
1,edrn/pgusordq,web_2
2,gygrt/cugfkt,web_3
3,gygrt/e2c,web_4
4,gygrt/iougz,web_5
5,gygrt/qodr,web_6
6,s/pgusordqqodr,web_7


In [25]:
# 合併客戶編號及網頁瀏覽, web_m3_select.drop(['Total'])
web_train = web_m3[['CUST_NO']].join(web_m3_select.drop(['Total']))
web_train = web_train.groupby(['CUST_NO']).sum().reset_index(level=0)
#轉換資料型態
# tb_m_list = tb_m.columns.tolist()[1:]
# tb_m[tb_m_list] = tb_m[tb_m_list].apply(lambda x:x.astype(int))

In [26]:
web_train.shape

(86867, 8)

##### **Output web CSV**

In [27]:
#output
web_train.to_csv('./data/web_train.csv', index = False)

### **3.2 Y_zero_use : 處理網址資料**
##### **篩出m3,m4, web1~web7網址瀏覽次數**

In [28]:
#篩出0~60天(m3,m4)的瀏覽資料,擷取網址
web_m4 = tb_web[tb_web['VISITDATE'] > 9507]
web_m4.reset_index(drop = True, inplace = True)

In [29]:
web_m4.shape

(848104, 6)

In [30]:
#做網址dummy，合併cust_no,計算客戶單一網址瀏覽次數
web_dummy_m4 = pd.get_dummies(web_m4['PAGE_M'])
#select web_1~web_7
web_m4_select = web_dummy_m4[page_m3]
#rename page
page_m4 = web_m4_select.columns.tolist()
m4_list = []
for i in range(len(web_m4_select.columns)):
    m4_list.append('web_' + str(i+1))
web_m4_select.columns = m4_list

In [31]:
page_pare = pd.DataFrame({'Page':page_m4, 'No.':m4_list})
page_pare.head(7)

,Page,No.
0,edrn/deoxt,web_1
1,edrn/pgusordq,web_2
2,gygrt/cugfkt,web_3
3,gygrt/e2c,web_4
4,gygrt/iougz,web_5
5,gygrt/qodr,web_6
6,s/pgusordqqodr,web_7


In [32]:
# 合併客戶編號及網頁瀏覽, 
web_test = web_m4[['CUST_NO']].join(web_m4_select)
web_test = web_test.groupby(['CUST_NO']).sum().reset_index(level=0)

##### **Output web CSV**

In [33]:
#output
web_test.to_csv('./data/web_test.csv', index = False)